In [2]:
import sys
sys.path.append('../')
import abc
import logging
import mne
import numpy as np
import pandas as pd
from brainModels.datasets.erpCoreP300 import ERPCORENP300
from brainModels.datasets.cogBciFlanker import COGBCIFLANKER
from brainModels.preprocessing.erp import ERP
from brainModels.featureExtraction.features import AutoRegressive
from brainModels.featureExtraction.features import PowerSpectralDensity
from brainModels.featureExtraction.twinNeural import TwinNeuralNetwork
from brainModels.datasets import utils
from sklearn.pipeline import make_pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from brainModels.evaluations.single_session_open_set import SingleSessionOpenSet
from brainModels.evaluations.multi_session_open_set import MultiSessionOpenSet
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC                                                                                 
from sklearn.ensemble import RandomForestClassifier
from brainModels.analysis.plotting import Plots 
import os

## Comparing the performace of single session and multi-session evaluations for datasets ERPCPREP300 and COG-BCI respectively.

### ERPCOREP300 

ERPCORENP300 is ERP based dataset and was generated as part of the study [1] E. S. Kappenman, J. L. Farrens, W. Zhang, A. X. Stewart, S. J. Luck, Erp core: An open resource for human event-related potential research, NeuroImage 225 (2021) 117465. 

| Name      | #Subj | #Chan | #Trials/class | Trials length | Sampling Rate | #Sessions |
|-----------|-------|-------|---------------|---------------|---------------|-----------|
| ERP: P300 | 40    | 32    | 5 NT x 1 T    | 1s            | 1024Hz        | 1         |

**Description of ERP Task in ERPCOREP300**

This ERP task Utilized an active 
visual oddball task to evoke the P300 response. Randomly presented letters A, B, C, D, and E
with equal probabilities (p = .2 for each letter). Designated one letter as the target 
for a block, and participants identified whether each stimulus was the target or a 
non-target. This approach maintained a constant target probability of .2, with the 
same physical stimulus serving as a target in some blocks and a non-target in others..

### COG-BCI 

COGBIC Flanker is a multi-session dataset and were generated as part of the studyies [1] Hinss, M. F., Jahanpour, E. S., Somon, B., Pluchon, L., Dehais, F., & Roy, R. N. (2022). COG-BCI database: A multi-session and multi-task EEG cognitive dataset for passive brain-computer interfaces (Version 1) [Data set]. Zenodo. https://doi.org/10.5281/zenodo.6874129 and [2] Lee, M. H., Kwon, O. Y., Kim, Y. J., Kim, H. K., Lee, Y. E., Williamson, J., … Lee, S. W. (2019). EEG dataset and OpenBMI toolbox for three BCI paradigms: An investigation into BCI illiteracy. GigaScience, 8(5), 1–16. https://doi.org/10.1093/gigascience/giz002 

| Name    | #Subj | #Chan | Sampling Rate | #Sessions |
|---------|-------|-------|---------------|-----------|
| COG-BCI |  29   | 32    |    512Hz      |    3      |

**Description of COG-BCI Flanker Task**

The dataset consists of recordings from 29 participants who completed three separate sessions, 
each conducted at an interval of 7 days. The participants are exposed to stimuli consisting 
of five arrows positioned at the center of a computer screen. Participants are instructed to 
respond to the central arrow while disregarding the surrounding (flanker) arrows. 
These flanker stimuli can aim in the same direction as the central target (congruent condition) 
or in the opposite direction (incongruent condition). 
Upon the conclusion of the trial, the participant is provided with feedback regarding 
the outcome of their performance, explicitly indicating whether their response was correct, 
incorrect, or a miss. A total of 120 trials are conducted, with each complete run having 
an approximate duration of 10 minutes. 

### Creating instances of datasets and ERP with default parameters

Following are the default parameters of dataset and paradigm ERP:</br></br>
    <i>Number of Subjects=40</i></br>
    <i>Sample_duration=1 seconds (-200,800)</i></br>
    <i>Baseline_Correction=True</i>

In [3]:
dataset1=ERPCORENP300()
dataset2=COGBCIFLANKER()
paradigm=ERP()

### Intializing pipelines for 4 shallow Classifiers i.e., LDA, LR, KNN and RF with AR Parameters and PSD Features with default Parameters

Following are the default parameters of AR coeffecients and Twin Neural Networks:</br></br>
    <i>AR Order=1</i></br>
    <i>Batch_Size=192</i></br>
    <i>Epochs=100</i></br>
    

In [4]:
pipeline={}
pipeline['AR+PSD+LDA']=make_pipeline(AutoRegressive(), PowerSpectralDensity(), LDA())
pipeline['AR+PSD+LR']=make_pipeline(AutoRegressive(), PowerSpectralDensity(), LogisticRegression())
pipeline['AR+PSD+KNN']=make_pipeline(AutoRegressive(), PowerSpectralDensity(), KNeighborsClassifier())
pipeline['AR+PSD+RF']=make_pipeline(AutoRegressive(), PowerSpectralDensity(), RandomForestClassifier())
pipeline['TNN']=make_pipeline(TwinNeuralNetwork())

### Creating Authentication pipeline for Single session evaluation under unkown attacker Scenario

In [5]:
single_evaluation=SingleSessionOpenSet(paradigm=paradigm, datasets=dataset1, overwrite=False)

### Executing the Single Session Authentication Pipeline

In [ ]:
Single_Session_results=single_evaluation.process(pipeline)

### Creating Authentication pipeline for Multi session evaluation under unkown attacker Scenario

In [ ]:
multi_evaluation=MultiSessionOpenSet(paradigm=paradigm, datasets=dataset2, overwrite=False)

### Executing the Multi Session Authentication Pipeline

In [ ]:
multi_session_results=multi_evaluation.process(pipeline)

### Concentenating the results of single session and multi session results

In [ ]:
# Concentenating the results of the two datasets
results=pd.concat([Single_Session_results, multi_session_results], axis=0)

### Statistical Analysis of the performace of Single and multi Session Evalaution Across the two datasets

Comparative Analysis of Average EER and FNMR at 1%, 0.1%, and 0.01% FMR thresholds across two datasets. The evaluation encompasses both single-session evaluation and multi-session evaluation Scheme, with a focus on classifiers’ performance in the unknown attacker scenario.

In [ ]:
results_df=results.groupby(['evaluation','eval Type','dataset','pipeline']).agg({
                'eer': lambda x: f'{np.mean(x)*100:.3f} ± {np.std(x)*100:.3f}',
                'frr_1_far': lambda x: f'{np.mean(x)*100:.3f}',
                'frr_0.1_far': lambda x: f'{np.mean(x)*100:.3f}',
                'frr_0.01_far': lambda x: f'{np.mean(x)*100:.3f}'    
            }).reset_index()

display(results_df)